In [1]:
from gym import spaces
import numpy as np
from environments.tic_tac_toe.tic_tac_toe import TicTacToe

In [7]:
test = TicTacToe()
test._grid[0][0] = 1
test._grid[1][0] = 1
test._grid[2][0] = 2
test.winner

0